### Tranform BigMac

In [1]:
# Import Modules / Dependencies:

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np

import sqlalchemy
import psycopg2
from sqlalchemy import create_engine

import datetime
from datetime import datetime

### THIS NEEDS TO BE IN WRITE UP
### Function to push the date to the beginning of the month or the end of the month
### Like excel to go to end of mo or beginning of mo

from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

In [3]:
df_path = "00_Extract_BigMacIndex_CSV.csv"
df_big_mac = pd.read_csv(df_path)
df_big_mac.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
0,4/1/2000,ARG,ARS,Argentina,2.50,1.00,2.500000,-0.00398,0.05007,-0.16722,-0.09864,1.09091,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/1/2000,AUS,AUD,Australia,2.59,1.68,1.541667,-0.38579,-0.35246,-0.48645,-0.44416,0.28939,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/1/2000,BRA,BRL,Brazil,2.95,1.79,1.648045,-0.34341,-0.30778,-0.45102,-0.40581,0.37836,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/1/2000,CAN,CAD,Canada,2.85,1.47,1.938776,-0.22758,-0.18566,-0.35417,-0.30099,0.62152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/1/2000,CHE,CHF,Switzerland,5.90,1.70,3.470588,0.38270,0.45774,0.15609,0.25130,1.90267,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_big_mac.tail()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
1325,1/14/2020,UKR,UAH,Ukraine,57.00,23.990,2.375990,-0.58095,-0.48146,-0.46125,-0.32961,-0.23913,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1326,1/14/2020,URY,UYU,Uruguay,179.00,37.435,4.781621,-0.15668,0.04355,0.08421,0.34915,0.53123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1327,1/14/2020,USA,USD,United States,5.67,1.000,5.670000,0.00000,0.23744,0.28565,0.59981,0.81572,62868.917,5.077074,0.000,0.028,0.091,0.318,0.092
1328,1/14/2020,VNM,VND,Vietnam,66000.00,23176.000,2.847774,-0.49775,-0.37849,-0.35428,-0.19649,-0.08805,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1329,1/14/2020,ZAF,ZAR,South Africa,31.00,14.391,2.154124,-0.62008,-0.52988,-0.51156,-0.39221,-0.31018,6353.846,2.930533,-0.342,-0.323,-0.282,-0.133,-0.281


In [5]:
### Get rid of the columns we dont need 
df_big_mac_reduced = df_big_mac[["date", "name", "dollar_ex"]]
df_big_mac_reduced.head()

,date,name,dollar_ex
0,4/1/2000,Argentina,1.00
1,4/1/2000,Australia,1.68
2,4/1/2000,Brazil,1.79
3,4/1/2000,Canada,1.47
4,4/1/2000,Switzerland,1.70


In [6]:
# Get China
df_big_mac_reduced_chyusd = df_big_mac_reduced.loc[df_big_mac_reduced["name"] == "China", :]
df_big_mac_reduced_chyusd.head()

,date,name,dollar_ex
6,4/1/2000,China,8.28
34,4/1/2001,China,8.28
62,4/1/2002,China,8.28
94,4/1/2003,China,8.28
125,5/1/2004,China,8.28


In [8]:
### Not sure if we need this - set date to index
df_big_mac_reduced_chyusd_2 = df_big_mac_reduced_chyusd.set_index('date')
df_big_mac_reduced_chyusd_2.head()

,name,dollar_ex
date,,
4/1/2000,China,8.28
4/1/2001,China,8.28
4/1/2002,China,8.28
4/1/2003,China,8.28
5/1/2004,China,8.28


In [11]:
### Now get me just the exchange rate we dont need the word China
df_big_mac_reduced_chyusd_3 = df_big_mac_reduced_chyusd_2[["dollar_ex"]]
df_big_mac_reduced_chyusd_3.head()

,dollar_ex
date,
4/1/2000,8.28
4/1/2001,8.28
4/1/2002,8.28
4/1/2003,8.28
5/1/2004,8.28


In [12]:
df_big_mac_reduced_chyusd_4 = df_big_mac_reduced_chyusd_3.rename(columns={"dollar_ex":"BigMac_CHYUSD"})
df_big_mac_reduced_chyusd_4.head()

,BigMac_CHYUSD
date,
4/1/2000,8.28
4/1/2001,8.28
4/1/2002,8.28
4/1/2003,8.28
5/1/2004,8.28


In [21]:

df_chy = df_big_mac_reduced_chyusd_4.reset_index()
df_chy

,date,BigMac_CHYUSD
0,4/1/2000,8.2800
1,4/1/2001,8.2800
2,4/1/2002,8.2800
3,4/1/2003,8.2800
4,5/1/2004,8.2800
5,6/1/2005,8.2764
6,1/1/2006,8.0641
7,5/1/2006,8.0260
8,1/1/2007,7.7743
9,6/1/2007,7.6047


In [17]:
df_big_mac_reduced_inrusd = df_big_mac_reduced.loc[df_big_mac_reduced["name"] == "India", :]
df_big_mac_reduced_inrusd.head()

,date,name,dollar_ex
530,7/1/2011,India,44.400
572,1/1/2012,India,51.905
613,7/1/2012,India,56.165
654,1/1/2013,India,53.395
696,7/1/2013,India,59.980


In [18]:
df_big_mac_reduced_inrusd_2 = df_big_mac_reduced_inrusd.set_index('date')
df_big_mac_reduced_inrusd_2.head()

,name,dollar_ex
date,,
7/1/2011,India,44.400
1/1/2012,India,51.905
7/1/2012,India,56.165
1/1/2013,India,53.395
7/1/2013,India,59.980


In [19]:
df_big_mac_reduced_inrusd_3 = df_big_mac_reduced_inrusd_2[["dollar_ex"]]
df_big_mac_reduced_inrusd_3.head()

,dollar_ex
date,
7/1/2011,44.400
1/1/2012,51.905
7/1/2012,56.165
1/1/2013,53.395
7/1/2013,59.980


In [20]:
df_big_mac_reduced_inrusd_4 = df_big_mac_reduced_inrusd_3.rename(columns={"dollar_ex":"BigMac_INRUSD"})
df_big_mac_reduced_inrusd_4.head()

,BigMac_INRUSD
date,
7/1/2011,44.400
1/1/2012,51.905
7/1/2012,56.165
1/1/2013,53.395
7/1/2013,59.980


In [22]:
df_inr = df_big_mac_reduced_inrusd_4.reset_index()
df_inr

,date,BigMac_INRUSD
0,7/1/2011,44.40000
1,1/1/2012,51.90500
2,7/1/2012,56.16500
3,1/1/2013,53.39500
4,7/1/2013,59.98000
5,1/1/2014,61.84500
6,7/1/2014,60.08875
7,1/1/2015,61.61500
8,7/1/2015,63.43000
9,1/1/2016,66.80250


### JAPAN

In [24]:
df_big_mac_reduced_jpyusd = df_big_mac_reduced.loc[df_big_mac_reduced["name"] == "Japan", :]
df_big_mac_reduced_jpyusd.head()

,date,name,dollar_ex
15,4/1/2000,Japan,106.0
42,4/1/2001,Japan,124.0
71,4/1/2002,Japan,130.0
103,4/1/2003,Japan,120.0
136,5/1/2004,Japan,113.0


In [25]:
df_big_mac_reduced_jpyusd_2 = df_big_mac_reduced_jpyusd.set_index('date')
df_big_mac_reduced_jpyusd_2.head()

,name,dollar_ex
date,,
4/1/2000,Japan,106.0
4/1/2001,Japan,124.0
4/1/2002,Japan,130.0
4/1/2003,Japan,120.0
5/1/2004,Japan,113.0


In [26]:
df_big_mac_reduced_jpyusd_3 = df_big_mac_reduced_jpyusd_2[["dollar_ex"]]
df_big_mac_reduced_jpyusd_3.head()

,dollar_ex
date,
4/1/2000,106.0
4/1/2001,124.0
4/1/2002,130.0
4/1/2003,120.0
5/1/2004,113.0


In [27]:
df_big_mac_reduced_jpyusd_4 = df_big_mac_reduced_jpyusd_3.rename(columns={"dollar_ex":"BigMac_JPYUSD"})
df_big_mac_reduced_jpyusd_4.head()

,BigMac_JPYUSD
date,
4/1/2000,106.0
4/1/2001,124.0
4/1/2002,130.0
4/1/2003,120.0
5/1/2004,113.0


In [28]:
df_jpy = df_big_mac_reduced_jpyusd_4.reset_index()
df_jpy

,date,BigMac_JPYUSD
0,4/1/2000,106.000
1,4/1/2001,124.000
2,4/1/2002,130.000
3,4/1/2003,120.000
4,5/1/2004,113.000
5,6/1/2005,106.715
6,1/1/2006,114.305
7,5/1/2006,112.105
8,1/1/2007,121.585
9,6/1/2007,122.315


### EURO

In [29]:
df_big_mac_reduced_eueusd = df_big_mac_reduced.loc[df_big_mac_reduced["name"] == "Euro area", :]
df_big_mac_reduced_eueusd.head()

,date,name,dollar_ex
9,4/1/2000,Euro area,1.075269
37,4/1/2001,Euro area,1.136364
65,4/1/2002,Euro area,1.123596
98,4/1/2003,Euro area,0.909091
131,5/1/2004,Euro area,0.833333


In [30]:
df_big_mac_reduced_eueusd_2 = df_big_mac_reduced_eueusd.set_index('date')
df_big_mac_reduced_eueusd_2.head()

,name,dollar_ex
date,,
4/1/2000,Euro area,1.075269
4/1/2001,Euro area,1.136364
4/1/2002,Euro area,1.123596
4/1/2003,Euro area,0.909091
5/1/2004,Euro area,0.833333


In [31]:
df_big_mac_reduced_eueusd_3 = df_big_mac_reduced_eueusd_2[["dollar_ex"]]
df_big_mac_reduced_eueusd_3.head()

,dollar_ex
date,
4/1/2000,1.075269
4/1/2001,1.136364
4/1/2002,1.123596
4/1/2003,0.909091
5/1/2004,0.833333


In [32]:
df_big_mac_reduced_eueusd_4 = df_big_mac_reduced_eueusd_3.rename(columns={"dollar_ex":"BigMac_EURUSD"})
df_big_mac_reduced_eueusd_4.head()

,BigMac_EURUSD
date,
4/1/2000,1.075269
4/1/2001,1.136364
4/1/2002,1.123596
4/1/2003,0.909091
5/1/2004,0.833333


In [33]:
df_eur = df_big_mac_reduced_eueusd_4.reset_index()
df_eur

,date,BigMac_EURUSD
0,4/1/2000,1.075269
1,4/1/2001,1.136364
2,4/1/2002,1.123596
3,4/1/2003,0.909091
4,5/1/2004,0.833333
5,6/1/2005,0.814930
6,1/1/2006,0.828157
7,5/1/2006,0.780366
8,1/1/2007,0.771813
9,6/1/2007,0.733756


### MERGE

In [34]:
df_combined_big_mac = pd.merge(df_chy, df_inr, on="date")
df_combined_big_mac.head()

,date,BigMac_CHYUSD,BigMac_INRUSD
0,7/1/2011,6.4450,44.400
1,1/1/2012,6.3155,51.905
2,7/1/2012,6.3885,56.165
3,1/1/2013,6.2204,53.395
4,7/1/2013,6.1341,59.980


In [35]:
df_combined_big_mac = pd.merge(df_combined_big_mac, df_jpy, on="date")
df_combined_big_mac.head()

,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD
0,7/1/2011,6.4450,44.400,78.365
1,1/1/2012,6.3155,51.905,76.920
2,7/1/2012,6.3885,56.165,78.220
3,1/1/2013,6.2204,53.395,91.065
4,7/1/2013,6.1341,59.980,100.105


In [36]:
df_combined_big_mac = pd.merge(df_combined_big_mac, df_eur, on="date")
df_combined_big_mac.head()

,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
0,7/1/2011,6.4450,44.400,78.365,0.697520
1,1/1/2012,6.3155,51.905,76.920,0.788239
2,7/1/2012,6.3885,56.165,78.220,0.824844
3,1/1/2013,6.2204,53.395,91.065,0.736947
4,7/1/2013,6.1341,59.980,100.105,0.777756


In [37]:
df_combined_big_mac

,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
0,7/1/2011,6.4450,44.40000,78.365,0.697520
1,1/1/2012,6.3155,51.90500,76.920,0.788239
2,7/1/2012,6.3885,56.16500,78.220,0.824844
3,1/1/2013,6.2204,53.39500,91.065,0.736947
4,7/1/2013,6.1341,59.98000,100.105,0.777756
5,1/1/2014,6.0513,61.84500,104.245,0.737218
6,7/1/2014,6.1983,60.08875,101.530,0.742749
7,1/1/2015,6.2115,61.61500,117.765,0.863073
8,7/1/2015,6.2092,63.43000,123.935,0.912700
9,1/1/2016,6.5582,66.80250,118.645,0.930233


### RESET

In [39]:
df_combined_big_mac_2 = df_combined_big_mac.reset_index()
df_combined_big_mac_2

,index,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
0,0,7/1/2011,6.4450,44.40000,78.365,0.697520
1,1,1/1/2012,6.3155,51.90500,76.920,0.788239
2,2,7/1/2012,6.3885,56.16500,78.220,0.824844
3,3,1/1/2013,6.2204,53.39500,91.065,0.736947
4,4,7/1/2013,6.1341,59.98000,100.105,0.777756
5,5,1/1/2014,6.0513,61.84500,104.245,0.737218
6,6,7/1/2014,6.1983,60.08875,101.530,0.742749
7,7,1/1/2015,6.2115,61.61500,117.765,0.863073
8,8,7/1/2015,6.2092,63.43000,123.935,0.912700
9,9,1/1/2016,6.5582,66.80250,118.645,0.930233


In [40]:
### Pushing all months to end of month

df_combined_big_mac_2['date'] = pd.to_datetime(df_combined_big_mac_2['date']) - MonthEnd(1) 


,index,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
0,0,2011-06-30,6.4450,44.40000,78.365,0.697520
1,1,2011-12-31,6.3155,51.90500,76.920,0.788239
2,2,2012-06-30,6.3885,56.16500,78.220,0.824844
3,3,2012-12-31,6.2204,53.39500,91.065,0.736947
4,4,2013-06-30,6.1341,59.98000,100.105,0.777756
5,5,2013-12-31,6.0513,61.84500,104.245,0.737218
6,6,2014-06-30,6.1983,60.08875,101.530,0.742749
7,7,2014-12-31,6.2115,61.61500,117.765,0.863073
8,8,2015-06-30,6.2092,63.43000,123.935,0.912700
9,9,2015-12-31,6.5582,66.80250,118.645,0.930233


In [44]:
df_c = df_combined_big_mac_2.drop('index', axis=1)
df_c

,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
0,2011-06-30,6.4450,44.40000,78.365,0.697520
1,2011-12-31,6.3155,51.90500,76.920,0.788239
2,2012-06-30,6.3885,56.16500,78.220,0.824844
3,2012-12-31,6.2204,53.39500,91.065,0.736947
4,2013-06-30,6.1341,59.98000,100.105,0.777756
5,2013-12-31,6.0513,61.84500,104.245,0.737218
6,2014-06-30,6.1983,60.08875,101.530,0.742749
7,2014-12-31,6.2115,61.61500,117.765,0.863073
8,2015-06-30,6.2092,63.43000,123.935,0.912700
9,2015-12-31,6.5582,66.80250,118.645,0.930233


In [51]:
df_c = df_c.rename(columns={"BigMac_CHYUSD":"bigmac_chyusd", "BigMac_INRUSD":"bigmac_inrusd", "BigMac_JPYUSD":"bigmac_jpyusd", "BigMac_EURUSD":"bigmac_eurusd"})
df_c

,date,bigmac_chyusd,bigmac_inrusd,bigmac_jpyusd,bigmac_eurusd
0,2011-06-30,6.4450,44.40000,78.365,0.697520
1,2011-12-31,6.3155,51.90500,76.920,0.788239
2,2012-06-30,6.3885,56.16500,78.220,0.824844
3,2012-12-31,6.2204,53.39500,91.065,0.736947
4,2013-06-30,6.1341,59.98000,100.105,0.777756
5,2013-12-31,6.0513,61.84500,104.245,0.737218
6,2014-06-30,6.1983,60.08875,101.530,0.742749
7,2014-12-31,6.2115,61.61500,117.765,0.863073
8,2015-06-30,6.2092,63.43000,123.935,0.912700
9,2015-12-31,6.5582,66.80250,118.645,0.930233


In [52]:
### df_c is the dataframe we want to push to DB
### End of transform big mac
df_c.to_csv("output/big_mac.csv", index = False)

#### AN ALTERNATE way because testing month begin

In [27]:
df_combined_big_mac_2['date'] = pd.to_datetime(df_combined_big_mac_2['date']) + MonthBegin(1) 
df_combined_big_mac_2

,date,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
0,2011-07-01,6.4450,44.40000,78.365,0.697520
1,2012-01-01,6.3155,51.90500,76.920,0.788239
2,2012-07-01,6.3885,56.16500,78.220,0.824844
3,2013-01-01,6.2204,53.39500,91.065,0.736947
4,2013-07-01,6.1341,59.98000,100.105,0.777756
5,2014-01-01,6.0513,61.84500,104.245,0.737218
6,2014-07-01,6.1983,60.08875,101.530,0.742749
7,2015-01-01,6.2115,61.61500,117.765,0.863073
8,2015-07-01,6.2092,63.43000,123.935,0.912700
9,2016-01-01,6.5582,66.80250,118.645,0.930233


In [28]:
df_combined_big_mac_2.dtypes

date             datetime64[ns]
BigMac_CHYUSD           float64
BigMac_INRUSD           float64
BigMac_JPYUSD           float64
BigMac_EURUSD           float64
dtype: object

In [29]:
df_combined_big_mac_2 = df_combined_big_mac_2.set_index('date')
df_combined_big_mac_2

,BigMac_CHYUSD,BigMac_INRUSD,BigMac_JPYUSD,BigMac_EURUSD
date,,,,
2011-07-01,6.4450,44.40000,78.365,0.697520
2012-01-01,6.3155,51.90500,76.920,0.788239
2012-07-01,6.3885,56.16500,78.220,0.824844
2013-01-01,6.2204,53.39500,91.065,0.736947
2013-07-01,6.1341,59.98000,100.105,0.777756
2014-01-01,6.0513,61.84500,104.245,0.737218
2014-07-01,6.1983,60.08875,101.530,0.742749
2015-01-01,6.2115,61.61500,117.765,0.863073
2015-07-01,6.2092,63.43000,123.935,0.912700
